In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# Guardrails with OpenAI

In [11]:
from nemoguardrails import LLMRails, RailsConfig

# load rails config
config = RailsConfig.from_path("./topic_rail_config/openai")

# create rails
rails = LLMRails(config, verbose=True)

In [12]:
res = await rails.generate_async(prompt="Hey there!")
print(res)

Hi there! How can I help you?
How are you doing today?


In [13]:
res = await rails.generate_async(prompt="Tell me about the latest soccer match.")
print(res)

I'm a shopping assistant, I don't like to talk of sports.
However, I can help you find the latest soccer merchandise or equipment. Is there anything specific you are looking for?


In [14]:
res = await rails.generate_async(prompt="Do you know who is Lionel Messi?")
print(res)

I'm a shopping assistant, I don't like to talk of sports.
However, I can help you with finding and purchasing any products you need.


# Guardrails with HuggingFace Pipeline - Llamav2

In [16]:
import huggingface_hub
huggingface_hub.login(token=os.environ["HUGGINGFACEHUB_API_TOKEN"])


from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/sofiaperez/.cache/huggingface/token
Login successful


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.90s/it]


In [18]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer,
    max_length=1024,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    logprobs=None,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

from nemoguardrails.llm.helpers import get_llm_instance_wrapper
from nemoguardrails.llm.providers import register_llm_provider


HFPipeline = get_llm_instance_wrapper(
    llm_instance=llm, llm_type="hf_pipeline_llama2"
)

register_llm_provider("hf_pipeline_llama2", HFPipeline)

In [27]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_path("./topic_rail_config/llama2")

# create rails
rails = LLMRails(config)


In [21]:
res = await rails.generate_async(prompt="hello")
print(res)

Parameter temperature does not exist for WrapperLLM


Hello! How can I assist you today?
How are you today?


In [28]:
res = await rails.generate_async(prompt="Tell me about the latest soccer match.")
print(res)

Parameter temperature does not exist for WrapperLLM


I'm a shopping assistant, I don't like to talk of sports.
Is there anything else I can help you with?


In [29]:
res = await rails.generate_async(prompt="Do you know who is Lionel Messi?")
print(res)

Parameter temperature does not exist for WrapperLLM


I'm a shopping assistant, I don't like to talk of sports.
Is there anything else I can help you with?
